In [1]:
library(brms)
theme_set(theme_default())

Loading required package: Rcpp
Loading required package: ggplot2
Loading 'brms' package (version 2.4.0). Useful instructions
can be found by typing help('brms'). A more detailed introduction
to the package is available through vignette('brms_overview').
Run theme_set(theme_default()) to use the default bayesplot theme.


# Load data

In [2]:
# load RData into environment object ("ex")
load("./prepped_data.RData", ex <- new.env())

In [3]:
# Load data from environment.
# For testing, we're using df_slice, which has only subjects 1 an 2.
#df <- ex$df_slice
df <- ex$df_total

In [4]:
head(df)

Answer,Block,CD,Choice,ED,RT,Trlnum,conNT,conNT_cent,item_id,newNT,newNT_cent,sub,task
1,0,1.140978,0,15.32376,1.762020,0,8.711341,-0.02945527,16.0,6.996014,0.002536885,1,disc
0,1,1.140978,1,15.32376,2.242939,33,8.711341,-0.02945527,16.0,6.996014,0.002536885,1,disc
1,2,1.140978,0,15.32376,1.766368,39,8.711341,-0.02945527,16.0,6.996014,0.002536885,1,disc
1,0,1.140978,0,15.32376,2.559490,38,8.711341,-0.02945527,16.0,6.996014,0.002536885,2,disc
1,2,1.140978,0,15.32376,2.417423,51,8.711341,-0.02945527,16.0,6.996014,0.002536885,2,disc
0,0,1.140978,1,15.32376,1.718840,2,8.711341,-0.02945527,16.0,6.996014,0.002536885,3,disc


In [5]:
str(df)

'data.frame':	11683 obs. of  14 variables:
 $ Answer    : Factor w/ 2 levels "0","1": 2 1 2 2 2 1 1 2 2 2 ...
 $ Block     : Factor w/ 3 levels "0","1","2": 1 2 3 1 3 1 2 3 1 2 ...
 $ CD        : num  1.14 1.14 1.14 1.14 1.14 ...
 $ Choice    : int  0 1 0 0 0 1 2 0 0 0 ...
 $ ED        : num  15.3 15.3 15.3 15.3 15.3 ...
 $ RT        : num  1.76 2.24 1.77 2.56 2.42 ...
 $ Trlnum    : int  0 33 39 38 51 2 51 46 31 6 ...
 $ conNT     : num  8.71 8.71 8.71 8.71 8.71 ...
 $ conNT_cent: num  -0.0295 -0.0295 -0.0295 -0.0295 -0.0295 ...
 $ item_id   : Factor w/ 64 levels "0.0","1.0","10.0",..: 9 9 9 9 9 9 9 9 9 9 ...
 $ newNT     : num  7 7 7 7 7 ...
 $ newNT_cent: num  0.00254 0.00254 0.00254 0.00254 0.00254 ...
 $ sub       : Factor w/ 32 levels "1","10","11",..: 1 1 1 12 12 23 23 23 27 27 ...
 $ task      : Factor w/ 2 levels "disc","name": 1 1 1 1 1 1 1 1 1 1 ...


# Modelling

## Overview

We want to predict accuracy ("Answer") by the following:

- Fixed effects structure:
    - $NT*Block$
    - NT can be: conNT, newNT, conNT AND newNT (incremental prediction)
    - $H_0$: Model without NT
    - We'll use the centered version of the neural typicality measures for interpretability

## Null models

In [6]:
# Build a null model, which assumes full random effects structure and a fixed effect for learning across blocks,
# but no influence of any neural typicality measure.
answer_nullmodel <- brm(Answer ~ Block + (Block|sub) + (1|item_id) + (1|task),
                        data = df,
                        family = bernoulli,
                        file = "answer_nullmodel",
                        chains = 2, cores = 2)

In [8]:
# Null model with varying slope for Block over tasks and over subjects
answer_nullmodel2 <- brm(Answer ~ Block + (Block|sub) + (Block|task) + (1|item_id),
                        data = df,
                        family = bernoulli,
                        file = "answer_nullmodel2",
                        chains = 2, cores = 2)

In [9]:
# Null model with random slope for block and task as well as their interaction across subjects
answer_nullmodel3 <- brm(Answer ~ Block*task + (Block*task|sub) + (1|item_id),
                        data = df,
                        family = bernoulli,
                        file = "answer_nullmodel3",
                        chains = 2, cores = 2)

In [26]:
# Null model equivalent to nullmodel3, but with random item slope over tasks.
#answer_nullmodel4 <- brm(Answer ~ Block*task + (Block*task|sub) + (item_id|task),
#                        data = df,
#                        family = bernoulli,
#                        file = "answer_nullmodel4",
#                        chains = 2, cores = 2)

In [35]:
# Null model 4 cannot be evaluated by stan

#system("cat nohup.out", intern=TRUE)

Compare null models:

In [27]:
waic_nullmodcomp <- waic(answer_nullmodel, answer_nullmodel2, answer_nullmodel3)

In [28]:
waic_nullmodcomp

                                          WAIC    SE
answer_nullmodel                      13557.19 94.08
answer_nullmodel2                     13562.06 93.97
answer_nullmodel3                     13519.29 94.84
answer_nullmodel - answer_nullmodel2     -4.86  1.49
answer_nullmodel - answer_nullmodel3     37.90 14.26
answer_nullmodel2 - answer_nullmodel3    42.77 14.16

Null model 3 fits best. (The next more complex model, number 4, could not be evaluated). Hence, We should also use its random effects structure for testing our models of interest.

### Newfound neural typicality

In [11]:
# Add the newfound neural typicality measure to the model
answer_newNT <- brm(Answer ~ newNT_cent*Block + (newNT_cent*Block|sub) + (1|item_id) + (1|task),
                    data = df,
                    family = bernoulli,
                    file = "answer_newNT",
                    chains = 2, cores = 2)

In [12]:
summary(answer_newNT)

Warning message:
“There were 95 divergent transitions after warmup. Increasing adapt_delta above 0.8 may help.
See http://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup”

 Family: bernoulli 
  Links: mu = logit 
Formula: Answer ~ newNT_cent * Block + (newNT_cent * Block | sub) + (1 | item_id) + (1 | task) 
   Data: df (Number of observations: 11683) 
Samples: 2 chains, each with iter = 2000; warmup = 1000; thin = 1;
         total post-warmup samples = 2000

Group-Level Effects: 
~item_id (Number of levels: 64) 
              Estimate Est.Error l-95% CI u-95% CI Eff.Sample Rhat
sd(Intercept)     0.95      0.09     0.80     1.15        279 1.01

~sub (Number of levels: 32) 
                                         Estimate Est.Error l-95% CI u-95% CI
sd(Intercept)                                0.54      0.08     0.40     0.72
sd(newNT_cent)                              18.19     10.58     0.82    39.86
sd(Block1)                                   0.25      0.08     0.08     0.41
sd(Block2)                                   0.29      0.08     0.15     0.45
sd(newNT_cent:Block1)                        7.19      5.77     0.35    20.92
sd(newNT_cent:Block2)

### Conserved neural typicality

In [13]:
answer_conNT <- brm(Answer ~ conNT_cent*Block + (conNT_cent*Block|sub) + (1|item_id) + (1|task),
                    data = df,
                    family = bernoulli,
                    file = "answer_conNT",
                    chains = 2, cores = 2)

In [14]:
summary(answer_conNT)

Warning message:
“There were 143 divergent transitions after warmup. Increasing adapt_delta above 0.8 may help.
See http://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup”

 Family: bernoulli 
  Links: mu = logit 
Formula: Answer ~ conNT_cent * Block + (conNT_cent * Block | sub) + (1 | item_id) + (1 | task) 
   Data: df (Number of observations: 11683) 
Samples: 2 chains, each with iter = 2000; warmup = 1000; thin = 1;
         total post-warmup samples = 2000

Group-Level Effects: 
~item_id (Number of levels: 64) 
              Estimate Est.Error l-95% CI u-95% CI Eff.Sample Rhat
sd(Intercept)     0.97      0.09     0.81     1.18        302 1.00

~sub (Number of levels: 32) 
                                         Estimate Est.Error l-95% CI u-95% CI
sd(Intercept)                                0.54      0.09     0.40     0.74
sd(conNT_cent)                               1.33      0.72     0.07     2.75
sd(Block1)                                   0.25      0.08     0.10     0.41
sd(Block2)                                   0.30      0.08     0.16     0.46
sd(conNT_cent:Block1)                        1.24      0.92     0.06     3.37
sd(conNT_cent:Block2)

### Newfound *AND* conserved typicality

In [15]:
answer_bothNT <- brm(Answer ~ conNT_cent*newNT_cent*Block + (conNT_cent*newNT_cent*Block|sub) + (1|item_id) + (1|task),
                    data = df,
                    family = bernoulli,
                    file = "answer_bothNT",
                    chains = 2, cores = 2)

In [16]:
#summary(answer_bothNT)

# Model comparison

In [22]:
# WAIC
waic_modcomp <- waic(answer_conNT, answer_newNT, answer_bothNT, answer_nullmodel)

In [23]:
print(waic_modcomp)

                                     WAIC    SE
answer_conNT                     13555.06 94.24
answer_newNT                     13545.20 94.20
answer_bothNT                    13537.82 94.64
answer_nullmodel                 13557.19 94.08
answer_conNT - answer_newNT          9.86 10.53
answer_conNT - answer_bothNT        17.25 10.35
answer_conNT - answer_nullmodel     -2.13  7.69
answer_newNT - answer_bothNT         7.38  9.30
answer_newNT - answer_nullmodel    -11.99  8.66
answer_bothNT - answer_nullmodel   -19.38 13.06


In [56]:
loo_modcomp <- loo(answer_conNT, answer_newNT, answer_bothNT, answer_nullmodel)

In [57]:
print(loo_modcomp)

                                    LOOIC    SE
answer_conNT                     13556.08 94.24
answer_newNT                     13546.16 94.21
answer_bothNT                    13539.24 94.66
answer_nullmodel                 13558.01 94.09
answer_conNT - answer_newNT          9.92 10.53
answer_conNT - answer_bothNT        16.83 10.36
answer_conNT - answer_nullmodel     -1.94  7.69
answer_newNT - answer_bothNT         6.92  9.31
answer_newNT - answer_nullmodel    -11.85  8.66
answer_bothNT - answer_nullmodel   -18.77 13.07


In [42]:
# retrieve individual fit measures

#waic_modcomp$answer_conNT$estimates

According to both WAIC and LOO, the order of model fit is:

- both NT
- new NT
- con NT
- null model

# Visualization

### Posteriors

In [47]:
# all posteriors
#plot(answer_newNT, ask = FALSE)

In [59]:
# Only regression weights
#plot(answer_bothNT, pars="^b")

### Model predictions

In [45]:
# check distribution of predictions
#pp_check(answer_newNT)

In [60]:
# Prediction by marginalized effects
#plot(marginal_effects(answer_newNT), ask = FALSE)

**Conclusions:**
- the posterior standard deviation for the interaction parameters have most of the mass close to zero.
            sd_sub__Block.Q:ED
            sd_sub__Block.L:ED
- This could indicate that letting them freely varying is not necessary or may even indicate overfitting. We should set up model(s) with these factors as only fixed effects and compare via LOO.

# TODO

## Interpretability

- Effect coding for categorical predictors (Block)

## Hypothesis testing

- Assess contrasts
- Reduce model complexity and compare with LOO

## Outlier evaluation

- Compare models based on data including and excluding outliers.